In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import yaml

from utils.datasets import create_dataloader, LoadImagesAndLabels
from utils.general import colorstr

/home/ammeling/projects/yolov7/utils/midog_dataset.py:18: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from models.yolo import Model

device = 'cuda:0'
hyp = yaml.load(open('data/hyp.scratch.p5.yaml', 'rb'), Loader=yaml.SafeLoader)  # load hyps'data/hyp.scratch.p5.yaml'

model = Model(
    cfg='cfg/training/yolov7.yaml',
    ch=3,
    nc=1
).to(device)
model.hyp = hyp
model.gr = 1.0

INFO:models.yolo:Overriding model.yaml nc=80 with nc=1
INFO:models.yolo:
                 from  n    params  module                                  arguments                     
INFO:models.yolo:  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
INFO:models.yolo:  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
INFO:models.yolo:  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
INFO:models.yolo:  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
INFO:models.yolo:  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
INFO:models.yolo:  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
INFO:models.yolo:  6                -1  1     36992  models.common.Conv     

In [4]:
hyp = yaml.load(open('data/hyp.scratch.p5.yaml', 'rb'), Loader=yaml.SafeLoader)
data_dict = yaml.load(open('data/coco.yaml'), Loader=yaml.SafeLoader)

In [5]:
train_path = data_dict['train']
imgsz = 640
batch_size = 4
gs = max(int(model.stride.max()), 32)  # grid size (max stride)
nl = model.model[-1].nl  # number of detection layers (used for scaling hyp['obj'])

# dl, ds = create_dataloader()

dataset = LoadImagesAndLabels(train_path, imgsz, batch_size, augment=False, 
                              hyp=hyp, rect=False, cache_images=False,  prefix=colorstr('train: '))

/home/ammeling/projects/yolov7/utils/datasets.py:412: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cache, exists = torch.load(cache_path), True  # load
train: Scanning '/da

In [6]:
from torch.utils.data import DataLoader

dl = DataLoader(dataset, batch_size, num_workers=2, collate_fn=dataset.collate_fn)

In [7]:
from utils.loss import ComputeLoss

compute_loss = ComputeLoss(model)


In [8]:
for batch in dl:
    images, targets, files, shapes = batch 
    break 

In [9]:
images.shape, targets.shape

(torch.Size([4, 3, 640, 640]), torch.Size([13, 6]))

In [10]:
targets

tensor([[0.00000e+00, 4.50000e+01, 4.79492e-01, 6.41578e-01, 9.55610e-01, 4.46625e-01],
        [0.00000e+00, 4.50000e+01, 7.36516e-01, 3.10391e-01, 4.98875e-01, 3.57313e-01],
        [0.00000e+00, 5.00000e+01, 6.37062e-01, 6.74703e-01, 4.94125e-01, 3.82937e-01],
        [0.00000e+00, 4.50000e+01, 3.39438e-01, 4.39172e-01, 6.78875e-01, 5.86125e-01],
        [0.00000e+00, 4.90000e+01, 6.46836e-01, 2.24414e-01, 1.18047e-01, 7.27033e-02],
        [0.00000e+00, 4.90000e+01, 7.73149e-01, 2.22352e-01, 9.07349e-02, 7.29221e-02],
        [0.00000e+00, 4.90000e+01, 6.68297e-01, 2.95180e-01, 1.31282e-01, 1.10172e-01],
        [0.00000e+00, 4.90000e+01, 6.42859e-01, 1.84414e-01, 1.48063e-01, 1.11047e-01],
        [1.00000e+00, 2.30000e+01, 7.70336e-01, 4.93140e-01, 3.35890e-01, 4.64313e-01],
        [1.00000e+00, 2.30000e+01, 1.85977e-01, 7.67320e-01, 2.06297e-01, 8.62345e-02],
        [2.00000e+00, 5.80000e+01, 5.19219e-01, 4.67312e-01, 3.98250e-01, 5.06437e-01],
        [2.00000e+00, 7.50000e+0

In [11]:
shapes

(((480, 640), ((1.0, 1.0), (0.0, 80.0))),
 ((426, 640), ((1.0, 1.0), (0.0, 107.0))),
 ((428, 640), ((1.0, 1.0), (0.0, 106.0))),
 ((425, 640), ((1.0, 1.0), (0.0, 107.5))))

In [19]:
pred = model(images.to(device))

RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [13]:
loss, loss_items = compute_loss(pred, targets.to(device))


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 12 but got size 3 for tensor number 1 in the list.

In [ ]:
import torch 

imgsz = 1024
shapes = imgsz * ds.shapes / ds.shapes.max(1, keepdims=True)
scale = np.random.uniform(0.9, 1.1, size=(shapes.shape[0], 1))  # augment scale
torch.tensor(np.concatenate([l[:, 3:5] * s for s, l in zip(shapes * scale, ds.labels)])).float()  # wh


tensor([[ 0.00000,  0.00000],
        [51.96932, 51.96932],
        [51.96932, 51.96932],
        ...,
        [53.04919, 53.04919],
        [53.04919, 53.04919],
        [ 0.00000,  0.00000]])